# New Section

In [3]:

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import datetime

import joblib
import pickle
import tempfile


import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']


In [4]:
model_data='s3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-10-08-59-02-189/output/model.tar.gz'

In [5]:
model_data

's3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-10-08-59-02-189/output/model.tar.gz'

In [6]:
from sagemaker import image_uris
image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [7]:
image=image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

In [8]:
image

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [9]:
role

'arn:aws:iam::479862333671:role/service-role/AmazonSageMaker-ExecutionRole-20220103T082598'

In [12]:
val = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val.csv'))

In [13]:
val

,demand,item_id,dept_id,cat_id,store_id,state_id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,date_day,date_week,date_month,date_year,date_week_m,lag_28,lag_35,lag_42,lag_49,lag_56,rolling_median_7,rolling_median_14,rolling_median_30,rolling_median_60,rolling_median_180
0,0,1437,3,1,0,0,1914,30,4,4,2,0,0,0,8.38,25,17,4,5,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1437,3,1,0,0,1915,30,4,4,2,0,0,0,8.38,26,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0,1437,3,1,0,0,1916,30,4,4,2,0,0,0,8.38,27,17,4,5,4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2,1437,3,1,0,0,1917,30,4,4,2,0,0,0,8.38,28,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0,1437,3,1,0,0,1918,30,4,4,2,0,0,0,8.38,29,17,4,5,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823225,1,1432,2,0,9,2,1940,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
823226,1,1433,2,0,9,2,1940,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
823227,0,1434,2,0,9,2,1940,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
823228,1,1435,2,0,9,2,1940,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [10]:
trainedmodel = sagemaker.model.Model(
    image_uri=image,
    model_data = model_data,
    role=role)

In [17]:
model_perdictor = trainedmodel.deploy(initial_instance_count=1, instance_type='ml.c5.4xlarge')

-----!

In [36]:
list(val.iloc[:,1:])

['item_id',
 'dept_id',
 'cat_id',
 'store_id',
 'state_id',
 'd',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'sell_price',
 'date_day',
 'date_week',
 'date_month',
 'date_year',
 'date_week_m',
 'lag_28',
 'lag_35',
 'lag_42',
 'lag_49',
 'lag_56',
 'rolling_median_7',
 'rolling_median_14',
 'rolling_median_30',
 'rolling_median_60',
 'rolling_median_180']

In [38]:
endpoint_name = 'sagemaker-xgboost-2022-01-10-11-06-00-228'


response = sagemaker_session.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'text/csv',
                                                Body = ','.join(map(str, val.iloc[:,1:].values[0])))


In [43]:
predictions = model_perdictor.predict(to_csv(val.iloc[:,1:])).decode('utf-8')

AttributeError: 'NoneType' object has no attribute 'predict'